In [41]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 81.5 ms, sys: 0 ns, total: 81.5 ms
Wall time: 121 ms


In [10]:
%%time
# Cấu hình Daskgateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()

client

An existing cluster was found. Connecting to: easihub.13058609f24c474aae81205b88f8e87c
CPU times: user 35.6 ms, sys: 4.93 ms, total: 40.5 ms
Wall time: 148 ms


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.13058609f24c474aae81205b88f8e87c/status,


In [5]:
# cấu hình thời gian lấy ảnh và tọa độ
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [6]:
data_sen2 = load_data_sen2(dc, date_range, coordinates)
display(data_sen2)


No datasets require offset correction
The valid_data_mask and scale (no offset) have been applied to the reflectance bands


<xarray.Dataset>
Dimensions:      (time: 151, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-02T03:35:23.960000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [18]:
data = np.load('datasets/gb_filled_cloud.npz')
fill_nan_model = joblib.load('model_train/regressor/500points/fill_nan_model_krf.pkl')

In [19]:
vh = data['vh']
vv = data['vv']
ndvi = data['ndvi']

In [11]:
ndvi.shape

(13, 8874, 9902)

In [20]:

coord_time = np.arange(13)
coord_x = data_sen2.coords['x']
coord_y = data_sen2.coords['y']


In [21]:
ndvi_xarray = xr.DataArray(ndvi, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vh_xarray = xr.DataArray(vh, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vv_xarray = xr.DataArray(vv, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})

In [ ]:
display(ndvi_xarray)

In [29]:
label_mapping = {
    "Lua tom": "1",
    "Lua": "3",
    "CHN": "4",
    "CLN": "5",
    "TS": "6",
    "Song": "7",
    "Dat xay dung": "8",
    "Rung": "9"
}


In [30]:
train_path = "train/ST_training data_updated_1130points.shp"
# train_path = "train/updated_data/ThuanHoa_DKS_Kappa.shp"
train = load_data_geo(train_path)

In [ ]:
train

In [32]:
def extract_data_point(train, average_ndvi, dsvh, dsvv):
    loaded_datasets = {}
    for idx, point in train.iterrows():
        key = f"point_{idx + 1}"
        try:
            ndvi_data = average_ndvi.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vh_data = dsvh.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vv_data = dsvv.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            loaded_datasets[key] = {
                "data": np.stack((ndvi_data, vh_data, vv_data), axis=1),
                "label": point.HT_code
            }
        except Exception as e:
            # Handle the exception if necessary
            print(f"Error at point {key}: {e}")
    return loaded_datasets

In [33]:
def create_dataset(datasets):
    data_points = []
    labels = []
    for point_key, point_data in datasets.items():
        data_array = point_data['data']
        label = point_data['label']
        for row in data_array:
            data_points.append(row)
            labels.append(label)
    return data_points, labels

In [34]:
# Extract data points and create the dataset
data_points = extract_data_point(train, ndvi_xarray, vh_xarray, vv_xarray)
X_train, y_train = create_dataset(data_points)

In [35]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [38]:
 X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [39]:
y_test

array([9, 7, 7, ..., 8, 4, 9])

In [40]:
print(X_train)

[[ 0.67069536  0.05427304  0.24030751]
 [ 0.79627126  0.05110675  0.23699677]
 [ 0.06312954  0.01186158  0.0612893 ]
 ...
 [-0.01124513  0.02648613  1.6112155 ]
 [ 0.4986132   0.02063359  0.07341269]
 [ 0.20368147  0.03412674  1.1948624 ]]


In [53]:
%%time
classifier, best_params  = cross_validate(
    train_data=(X_train, y_train), 
    model_class=classifiers['svm'], 
    param_grid=param_grids_classifier['svm'], 
    metric='accuracy',  # Use the correct scoring for MSE
    num_fold=5
)
print(best_params)

Average accuracy: 0.716176312479072
Best accuracy score: 0.7314893617021276
{'model__C': 10, 'model__kernel': 'rbf'}
CPU times: user 19 s, sys: 36 µs, total: 19 s
Wall time: 35.7 s


In [54]:
y_pred_test = classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for test data {round(test_accuracy, 2)*100} %")

Accuracy for test data 71.0 %


In [52]:
save_model("new_classify/classify_svm.joblib", classifier)

Done!
